<a href="https://colab.research.google.com/github/nrohrbach/PocDatenkatalogBafu/blob/main/PoC_Datenkatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test
Daten: http://www.bafu-daten.ch/wasser/restwasser/data/data/index/rawData_d.txt

In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [22]:
Options = ['Abfall',
           'Altlasten',
           'Bildung, Forschung, Innovation',
           'Biodiversität',
           'Biotechnologie',
           'Boden',
           'Chemikalien',
           'Elektrosmog und Licht',
           'Ernährung, Wohnen, Mobilität',
           'Gesundheit',
           'Internationales',
           'Klima',
           'Landschaft',
           'Lärm',
           'Luft',
           'Naturgefahren',
           'Recht',
           'Störfallvorsorge',
           'Umweltverträglichkeitsprüfung',
           'Wald & Holz',
           'Wasser',
           'Wirtschaft und Konsum'
           ]

In [20]:
def map_options(option):
  mapping = {
      'Abfall': 'abfall',
      'Altlasten': 'altlasten',
      'Bildung, Forschung, Innovation': 'bildung',
      'Biodiversität': 'biodiversitat',
      'Biotechnologie' : 'biotechnologie',
      'Boden' : 'boden',
      'Chemikalien' : 'chemikalien',
      'Elektrosmog und Licht' : 'elektrosmog',
      'Ernährung, Wohnen, Mobilität'  : 'ernaehrung',
      'Gesundheit'  : 'gesundheit',
      'Internationales'  : 'internationales',
      'Klima'  : 'klima',
      'Landschaft'  : 'landschaft',
      'Lärm'  : 'laerm',
      'Luft'  : 'luft',
      'Naturgefahren'  : 'naturgefahren',
      'Recht'  : 'recht',
      'Störfallvorsorge'  : 'storfallvorsorge',
      'Umweltverträglichkeitsprüfung'  : 'umweltvertraeglichkeitpruefung',
      'Wald & Holz'  : 'wald',
      'Wasser'  : 'wasser',
      'Wirtschaft und Konsum'  : 'wirtschaft'
  }
  return mapping.get(option, None) # Return None if option is not found


In [14]:
# Funktion fragt alle Datensätze von opendata.swiss nach einem Keyword ab

def search_datasets(keyword):
    # Basis-URL der CKAN API
    base_url = "https://opendata.swiss/api/3/action/package_search"

    # Parameter für die Abfrage
    params = {
        "fq": f"organization:{'bundesamt-fur-umwelt-bafu'}",
        "q": f"keywords_de:{keyword}"
    }

    # Anfrage an die API senden
    response = requests.get(base_url, params=params)
    data = response.json()
    data = data["result"]["results"]

    # Daten aus Json auslesen
    description = [s['description']['de'] for s in data]
    title = [s['title']['de'] for s in data]
    url0 = [s['resources'][0]['url'] for s in data]
    title0 = [s['resources'][0]['title']['de'] for s in data]
    url1 = [s['resources'][1]['url'] for s in data]
    title1 = [s['resources'][1]['title']['de'] for s in data]

    # Daten als Dataframe speichern
    dict = {'title': title, 'description': description, 'url0': url0, 'title0': title0, 'url1': url1, 'title1': title1}
    dfOpendataSwiss = pd.DataFrame(dict)


    url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser")
    table_body = soup.find("table", attrs={"class": "table"}) # Corrected table identification

    if table_body:
        rows = table_body.find_all("tr")
        data = []
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2:  # Ensure there are at least two cells
                thema = cells[0].text.strip()
                indikatorname = cells[1].text.strip()
                # Extract href from the second cell (index 1)
                link_element = cells[1].find('a')
                if link_element:
                    href = link_element.get('href')
                    data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch"+ href})
                else:
                    data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": None}) # Handle cases where no link is found.


        df = pd.DataFrame(data)
    df



    return dfOpendataSwiss

In [23]:
search_datasets('luft')

,title,description,url0,title0,url1,title1
0,Luftbelastung entlang der A2 und der A13 - Jah...,"Das Projekt ""Monitoring flankierende Massnahme...",https://visualize.admin.ch/en/browse?dataset=h...,visualize.admin.ch,https://lindas.admin.ch/sparql#query=PREFIX+sc...,SPARQL Endpoint mit Vorauswahl des Graph
1,Luftbelastung durch Ozon,Bodennahes Ozon hat negative Auswirkungen auf ...,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...,ch.bafu.luftreinhaltung-ozon,https://wmts.geo.admin.ch/EPSG/3857/1.0.0/WMTS...,ch.bafu.luftreinhaltung-ozon
2,Luftbelastung durch Stickstoffdioxid,Unter dem Begriff Stickoxide (NOx) werden Stic...,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...,ch.bafu.luftreinhaltung-stickstoffdioxid,https://wmts.geo.admin.ch/EPSG/3857/1.0.0/WMTS...,ch.bafu.luftreinhaltung-stickstoffdioxid
3,Luftbelastung durch Feinstaub PM2.5,"Unter PM2.5 versteht man Feinstaubpartikel, di...",https://data.geo.admin.ch/browser/index.html#/...,STAC Browser,https://www.bafu.admin.ch/bafu/de/home/zustand...,BAFU Website
4,Luftbelastung durch Schwefeldioxid,Höhere Belastungen von Schwefeldioxid (SO₂) kö...,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...,ch.bafu.luftreinhaltung-schwefeldioxid,https://wmts.geo.admin.ch/EPSG/3857/1.0.0/WMTS...,ch.bafu.luftreinhaltung-schwefeldioxid


In [24]:
url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes

soup = BeautifulSoup(response.content, "html.parser")
table_body = soup.find("table", attrs={"class": "table"}) # Corrected table identification

if table_body:
    rows = table_body.find_all("tr")
    data = []
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 2:  # Ensure there are at least two cells
            thema = cells[0].text.strip()
            indikatorname = cells[1].text.strip()
            # Extract href from the second cell (index 1)
            link_element = cells[1].find('a')
            if link_element:
                href = link_element.get('href')
                data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch"+ href})
            else:
                data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": None}) # Handle cases where no link is found.


    df = pd.DataFrame(data)


In [28]:
# prompt: Folgende Abfrage als Funktion beschreiben. Input ist das Thema. Die Funktion filtert df nach Thema

def filter_df_by_thema(thema):
    """
    Filters the DataFrame df based on the provided Thema.

    Args:
        thema: The Thema to filter by.

    Returns:
        A DataFrame containing only the rows where the 'Thema' column matches the input thema.
        Returns None if the input 'thema' is invalid, or if the dataframe is not created
        successfully.
    """

    url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

    try:
      soup = BeautifulSoup(response.content, "html.parser")
      table_body = soup.find("table", attrs={"class": "table"})

      if not table_body:
        print("Table not found on the webpage")
        return None

      rows = table_body.find_all("tr")
      data = []
      for row in rows:
          cells = row.find_all("td")
          if len(cells) >= 2:
              thema_cell = cells[0].text.strip()
              indikatorname = cells[1].text.strip()
              link_element = cells[1].find('a')
              href = link_element.get('href') if link_element else None
              data.append({"Thema": thema_cell, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch" + href if href else None})

      df = pd.DataFrame(data)
    except Exception as e:
      print(f"An error occurred while processing the webpage: {e}")
      return None

    # Filter the DataFrame using .isin() for multiple Thema values if needed
    if isinstance(thema, list):
      filtered_df = df[df["Thema"].isin(thema)]
    else:
      filtered_df = df[df["Thema"] == thema]

    return filtered_df


In [29]:
filter_df_by_thema('Luft')

,Thema,Indikatorname,Link
61,Luft,Ammoniak-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
62,Luft,Benzol-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
63,Luft,Feinstaub-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
64,Luft,Feinstaubimmissionen (PM2.5) in städtischer Um...,https://www.bafu.admin.ch/bafu/de/home/themen/...
65,Luft,Gesundheitskosten der Luftverschmutzung,https://www.bafu.admin.ch/bafu/de/home/themen/...
66,Luft,Gesundheitsschäden durch Luftverschmutzung,https://www.bafu.admin.ch/bafu/de/home/themen/...
67,Luft,Ozon-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
68,Luft,Schwefeldioxid-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
69,Luft,Stickstoffdioxid-Immissionen,https://www.bafu.admin.ch/bafu/de/home/themen/...
70,Luft,Subjektive Belastung durch Luftverschmutzung,https://www.bafu.admin.ch/bafu/de/home/themen/...
